In [2]:
%matplotlib inline
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from scipy.spatial.distance import cdist
from scipy import stats
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("MNIST_data/")

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


In [3]:
X_train = mnist.train.images
X_test = mnist.test.images
Y_train = mnist.train.labels
Y_test = mnist.test.labels

In [4]:
def one_hot_encode(y):
    encoded = np.zeros((len(y), np.max(y)+1), dtype="uint8")
    encoded[np.arange(len(y)), y] = 1
    assert len(y) == encoded.shape[0]    
    return encoded


print(one_hot_encode(np.r_[5,6,7]))
print('\n')
print(one_hot_encode(np.r_[5,6,7,5]))

[[0 0 0 0 0 1 0 0]
 [0 0 0 0 0 0 1 0]
 [0 0 0 0 0 0 0 1]]


[[0 0 0 0 0 1 0 0]
 [0 0 0 0 0 0 1 0]
 [0 0 0 0 0 0 0 1]
 [0 0 0 0 0 1 0 0]]


In [5]:
def one_hot_decode(y):
    y = np.argmax(y, axis=1)
    assert(len(y) == y.shape[0])
    return y

In [6]:
Y_train2 = one_hot_encode(Y_train)
Y_test2 = one_hot_encode(Y_test)

# add a column of 1s as the new first column in X_train and X_test as the bias term 'c' in y = mx + c
X_train2 = np.insert(X_train, 0, 1, axis=1)
print(X_train2.shape)
X_test2 = np.insert(X_test, 0, 1, axis=1)
print(X_test2.shape)

assert(np.all(X_train2[:,0] == 1.0))
assert(X_train2.shape[1] == X_train.shape[1]+1)

assert(np.all(X_test2[:,0] == 1.0))
assert(X_test2.shape[1] == X_test.shape[1]+1)

(55000, 785)
(10000, 785)


In [7]:
# test cases:
assert np.all(one_hot_decode(one_hot_encode(Y_train)) == Y_train)
assert np.all(one_hot_decode(one_hot_encode(Y_test)) == Y_test)

In [7]:
import tensorflow as tf

In [8]:
#declare variables

X = tf.placeholder(tf.float32, [None,785]) #X_train2, X_test2
Y = tf.placeholder(tf.float32, [None,10]) #Y_train2, Y_test2
A = tf.Variable(tf.random_normal((785,10))) # X @ A = Y 

In [9]:
A

<tf.Variable 'Variable:0' shape=(785, 10) dtype=float32_ref>

In [10]:
X

<tf.Tensor 'Placeholder:0' shape=(?, 785) dtype=float32>

In [10]:
#Y_pred = softmax(X_train2 @ A)

Y_pred = tf.nn.softmax(tf.matmul(X,A))

In [11]:
Y_pred

<tf.Tensor 'Softmax:0' shape=(?, 10) dtype=float32>

In [12]:
#     Y_pred = softmax(X_train2@A) # Y_pred.shape == (n,10)
#     return -np.sum(Y_train2*np.log(Y_pred))/X_train2.shape[0]

cross_entropy =  -tf.reduce_sum(Y*tf.log(Y_pred))/tf.cast(tf.shape(X), tf.float32)[0]
cross_entropy

<tf.Tensor 'truediv:0' shape=() dtype=float32>

In [13]:
#np.mean(np.argmax(Y_pred, axis=1) == np.argmax(Y_train2, axis=1))
accuracy = tf.reduce_mean(
    tf.cast(
        tf.equal(tf.arg_max(Y_pred,1 ), tf.arg_max(Y, 1) ),
        tf.float32
    )
)

Instructions for updating:
Use `argmax` instead


In [14]:
eta = 0.5 #learning rate
#A = A - eta *  cross_entropy_grad(A,X,Y)
train_step = tf.train.GradientDescentOptimizer(eta).minimize(cross_entropy)

In [15]:
train_step

<tf.Operation 'GradientDescent' type=NoOp>

In [17]:
sess = tf.Session()
sess.run(tf.global_variables_initializer())
maxiter = 10000
subset_size = X_train2.shape[0] // 100

for i in range(maxiter):
    #generate subset_size random indexes
    idx = np.random.randint(X_train2.shape[0], size=subset_size)
    sess.run(train_step, feed_dict={
         X: X_train2[idx,:],
         Y: Y_train2[idx,:]   
    })

print(sess.run(accuracy,feed_dict={
         X: X_train2,
         Y: Y_train2   
    }))    

0.922618


In [19]:
A1 = tf.Variable(tf.random_normal( (785, 47) ))
A2 = tf.Variable(tf.random_normal( (47, 10) ))
Y_pred = tf.nn.softmax(
    tf.matmul(tf.nn.relu(tf.matmul(X, A1)), A2)
)

cross_entropy = -tf.reduce_sum(Y * tf.log(Y_pred))/tf.cast(tf.shape(X), tf.float32)[0]
accuracy = tf.reduce_mean(
    tf.cast(
        tf.equal(tf.argmax(Y_pred, 1), tf.argmax(Y, 1))
    , tf.float32)
)
train_step = tf.train.GradientDescentOptimizer(eta).minimize(cross_entropy)

sess = tf.Session()
sess.run(tf.global_variables_initializer())
maxiter = 10000
subset_size = X_train2.shape[0] // 100
for i in range(maxiter):
    # generate subset_size random indexes
    idx = np.random.randint(X_train2.shape[0], size=subset_size)
    sess.run(train_step, feed_dict={
        X: X_train2[idx,:],
        Y: Y_train2[idx,:]
    })

print(sess.run(accuracy, feed_dict={
        X: X_train2,
        Y: Y_train2
    }))

0.0989818


In [17]:
import keras

Using TensorFlow backend.


In [18]:
keras.__version__
tf.__version__

'1.8.0'

In [20]:
from keras.models import Sequential
from keras.layers import Dense, Activation

model = Sequential()
model.add(Dense(10, input_dim=784))

model.add(Activation("softmax"))
model.compile(loss="categorical_crossentropy", optimizer="sgd", metrics=["accuracy"])
model.fit(X_train, Y_train2, epochs=10, batch_size=100, verbose=1 )

Epoch 1/10
55000/55000 [==============================] - 1s 15us/step - loss: 1.6848 - acc: 0.4781
Epoch 2/10
55000/55000 [==============================] - 1s 13us/step - loss: 0.9210 - acc: 0.7593
Epoch 3/10
55000/55000 [==============================] - 1s 13us/step - loss: 0.6492 - acc: 0.8195
Epoch 4/10
55000/55000 [==============================] - 1s 13us/step - loss: 0.4886 - acc: 0.8661
Epoch 5/10
55000/55000 [==============================] - 1s 13us/step - loss: 0.4293 - acc: 0.8806
Epoch 6/10
55000/55000 [==============================] - 1s 13us/step - loss: 0.3991 - acc: 0.8876
Epoch 7/10
55000/55000 [==============================] - 1s 13us/step - loss: 0.3795 - acc: 0.8933
Epoch 8/10
55000/55000 [==============================] - 1s 13us/step - loss: 0.3655 - acc: 0.8970
Epoch 9/10
55000/55000 [==============================] - 1s 13us/step - loss: 0.3544 - acc: 0.9000
Epoch 10/10
55000/55000 [==============================] - 1s 13us/step - loss: 0.3455 - acc: 0.9031

In [23]:
from keras.models import Sequential
from keras.layers import Dense, Activation

model = Sequential()
model.add(Dense(10, input_dim=784))
model.add(Dense(10, activation="relu"))
model.add(Activation("softmax"))
model.compile(loss="categorical_crossentropy", optimizer="sgd", metrics=["accuracy"])
model.fit(X_train, Y_train2, epochs=10, batch_size=100, verbose=1 )

Epoch 1/10
55000/55000 [==============================] - 1s 16us/step - loss: 1.6110 - acc: 0.5153
Epoch 2/10
55000/55000 [==============================] - 1s 13us/step - loss: 1.0335 - acc: 0.6929
Epoch 3/10
55000/55000 [==============================] - 1s 13us/step - loss: 0.9090 - acc: 0.7112
Epoch 4/10
55000/55000 [==============================] - 1s 14us/step - loss: 0.8506 - acc: 0.7202
Epoch 5/10
55000/55000 [==============================] - 1s 13us/step - loss: 0.8159 - acc: 0.7254
Epoch 6/10
55000/55000 [==============================] - 1s 12us/step - loss: 0.7928 - acc: 0.7297
Epoch 7/10
55000/55000 [==============================] - 1s 12us/step - loss: 0.7763 - acc: 0.7330
Epoch 8/10
55000/55000 [==============================] - 1s 12us/step - loss: 0.7638 - acc: 0.7354
Epoch 9/10
55000/55000 [==============================] - 1s 12us/step - loss: 0.7540 - acc: 0.7368
Epoch 10/10
55000/55000 [==============================] - 1s 13us/step - loss: 0.7460 - acc: 0.7382